# Машинное обучение ранжированию с использованием библиотеки CatBoost

В этом примере мы:
- увидим как выглядят датасеты для машинного обучения ранжированию, на примере стандартного датасета [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- познакомимся с библиотекой **CatBoost**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **CatBoost**

## Подготовка

Импортируем все что нам понадобится для дальнейшей работы:

In [3]:
import copy
from timeit import default_timer as timer

import catboost
from catboost import datasets, utils

## Скачиваем датасет MSLR (Microsoft Learning to Rank)

Дальше мы будем работать с датасетом MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Мы этого делать не будем т.к. в CatBoost уже встроена возможность загрузить небольшой сабсет MSLR, с которым мы и будем работать дальше.

Загрузим этот сабсет:

In [4]:
df_train, df_test = catboost.datasets.msrank_10k()

Датасет представляет собой обычный табличный датасет из 130 колонок:
- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит query_id, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (значения BM25 и т.п.), их точная природа нам сейчас на важна

Посмотрим на данные:

In [5]:
print(df_train.head(10))

   0    1    2    3    4    5    6    7    8         9    ...  128       129  \
0  2.0    1    3    3    0    0    3  1.0  1.0  0.000000  ...   62  11089534   
1  2.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   54  11089534   
2  0.0    1    3    0    2    0    3  1.0  0.0  0.666667  ...   45         3   
3  2.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   56  11089534   
4  1.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   64         5   
5  1.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   62         6   
6  1.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   62         6   
7  2.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   73         0   
8  1.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   51  11089534   
9  0.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   48         1   

   130  131    132  133  134  135  136  137  
0    2  116  64034   13    3    0    0  0.0  
1    2  124  64034    1    

Для удобства присвоим колонкам говорящие имена:

In [6]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят красивее:

In [7]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0    1          3          3          0          0          3   
1    2.0    1          3          0          3          0          3   
2    0.0    1          3          0          2          0          3   
3    2.0    1          3          0          3          0          3   
4    1.0    1          3          0          3          0          3   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...           62     11089534   
1        1.0        0.0   1.000000  ...           54     11089534   
2        1.0        0.0   0.666667  ...           45            3   
3        1.0        0.0   1.000000  ...           56     11089534   
4        1.0        0.0   1.000000  ...           64            5   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0            2          116        64034           13            3   
1           

Проведем небольшой EDA.

Всего у нас 10000 документов в трейне:

In [8]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 138 entries, label to feature_136
dtypes: float64(97), int64(41)
memory usage: 10.5 MB
None


И 10000 документов в тесте:

In [9]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 138 entries, label to feature_136
dtypes: float64(97), int64(41)
memory usage: 10.5 MB
None


Сколько у нас всего запросов?

In [10]:
num_queries_train = df_train['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train and {num_queries_test} test queries")

Got 87 train and 88 test queries


Получается, у примерно по 100 документов на запрос.

Это типично, когда, например, для сбора датасета обкачивались и заливались на оценку топ-100 документов поисковой выдачи по случайным запросам.

Какие у нас таргеты (оценки?):

In [11]:
print(df_train['label'].value_counts())

label
0.0    5481
1.0    3000
2.0    1326
3.0     142
4.0      51
Name: count, dtype: int64


In [12]:
print(df_test['label'].value_counts())

label
0.0    5755
1.0    2830
2.0    1221
3.0     148
4.0      46
Name: count, dtype: int64


Конвертируем датасет в формате, который можно подавать на вход модели:

In [13]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_catboost_dataset(df_train)
X_test, y_test, q_test = to_catboost_dataset(df_test)

Нормализуем оценки релевантностей, и попутно запомним исходной вектор таргетов чтобы потом использовать его для расчета метрик качества:

In [14]:
y_max = 4
y_train /= y_max
y_test_orig = y_test.copy()
y_test /= y_max

Подготовим пулы:

In [15]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

Зададим целевую метрику, которую будем пытаться оптимизировать.  
В нашем случае будем использовать NDCG@10

In [16]:
EVAL_METRIC = 'NDCG:top=10;type=Exp'

Параметры обучения модели:
- зададим целевую метрику
- фиксируем сид генератора случайных чисел
- зададим число итераций, после которого останавливаем обучение если в течение данного числа итераций мы не наблюдаем улучшения целевой метрики на валидационном множесте

Для обучения на GPU надо добавить параметр *task_type: GPU*

In [17]:
DEFAULT_PARAMS = {
    'iterations': 2000,             # maximum number of trees
    'early_stopping_rounds': 200,   # stop if metric does not improve for N rounds
    'eval_metric': EVAL_METRIC,
    'random_seed': 22,
    'verbose': 10
}

Мы будем обучать разные модели, использующие разные лоссы, соответствующие разным алгоритмам Learning to Rank.  
Напишем функцию, которая позволит кастомизировать модель под нужный лосс:

In [18]:
def create_model(loss_function):
    params = copy.deepcopy(DEFAULT_PARAMS)

    # Temporary that is used by catboost to store additional information
    catboost_info_dir = f"/tmp/catboost_info.{loss_function.lower()}"

    params.update({
        'loss_function': loss_function,
        'train_dir': str(catboost_info_dir),
    })
    return catboost.CatBoost(params)

Создадим и обучим pointwise модель которая в качестве лосса используем обычное RMSE:

In [19]:
# Create model
model = create_model('RMSE')

# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"model fit: elapsed = {elapsed:.3f} num_trees = {model.tree_count_}")

# Save model
# model.save_model("/tmp/model.cbm")

Learning rate set to 0.047893
0:	test: 0.2836948	best: 0.2836948 (0)	total: 87.9ms	remaining: 2m 55s
10:	test: 0.4296218	best: 0.4296218 (10)	total: 168ms	remaining: 30.5s
20:	test: 0.4432635	best: 0.4436566 (19)	total: 245ms	remaining: 23.1s
30:	test: 0.4443193	best: 0.4471698 (29)	total: 318ms	remaining: 20.2s
40:	test: 0.4501459	best: 0.4522379 (36)	total: 392ms	remaining: 18.7s
50:	test: 0.4568131	best: 0.4568131 (50)	total: 464ms	remaining: 17.7s
60:	test: 0.4604295	best: 0.4609420 (59)	total: 537ms	remaining: 17.1s
70:	test: 0.4570174	best: 0.4609420 (59)	total: 607ms	remaining: 16.5s
80:	test: 0.4586053	best: 0.4609420 (59)	total: 681ms	remaining: 16.1s
90:	test: 0.4623113	best: 0.4644785 (88)	total: 752ms	remaining: 15.8s
100:	test: 0.4656290	best: 0.4656290 (100)	total: 821ms	remaining: 15.4s
110:	test: 0.4639706	best: 0.4664429 (101)	total: 894ms	remaining: 15.2s
120:	test: 0.4674214	best: 0.4677040 (119)	total: 971ms	remaining: 15.1s
130:	test: 0.4688286	best: 0.4706122 (128

Получим предикты модели на тестовом множестве:

In [20]:
y_hat_test = model.predict(pool_test)
print("got predictions")

got predictions


Посчиатем метрики качества:

In [21]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp', 'MAP:top=10']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test using original (non-normalized) labels
compute_metrics(y_test_orig, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.427
metric = DCG:top=10;type=Exp score = 8.766
metric = MAP:top=10 score = 0.535


Теперь проделаем все то же самое, но с использованием алгоритма YetiRank:

In [24]:
# Create model
model = create_model('YetiRank')

# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"model fit: elapsed = {elapsed:.3f} num_trees = {model.tree_count_}")

# Predict
y_hat_test = model.predict(pool_test)

# Compute metrics on test using original (non-normalized) labels
print("\nevaluated:")
compute_metrics(y_test_orig, y_hat_test, q_test)

0:	test: 0.3331801	best: 0.3331801 (0)	total: 48.4ms	remaining: 1m 36s
10:	test: 0.4529994	best: 0.4529994 (10)	total: 231ms	remaining: 41.7s
20:	test: 0.4559417	best: 0.4566992 (12)	total: 398ms	remaining: 37.5s
30:	test: 0.4501092	best: 0.4574508 (22)	total: 573ms	remaining: 36.4s
40:	test: 0.4658935	best: 0.4658935 (40)	total: 740ms	remaining: 35.3s
50:	test: 0.4646162	best: 0.4693073 (45)	total: 900ms	remaining: 34.4s
60:	test: 0.4674726	best: 0.4700926 (58)	total: 1.08s	remaining: 34.4s
70:	test: 0.4718160	best: 0.4718160 (70)	total: 1.28s	remaining: 34.7s
80:	test: 0.4738091	best: 0.4738091 (80)	total: 1.44s	remaining: 34.2s
90:	test: 0.4735125	best: 0.4762371 (82)	total: 1.6s	remaining: 33.6s
100:	test: 0.4729083	best: 0.4762371 (82)	total: 1.77s	remaining: 33.3s
110:	test: 0.4700168	best: 0.4762371 (82)	total: 1.93s	remaining: 32.8s
120:	test: 0.4838545	best: 0.4838545 (120)	total: 2.09s	remaining: 32.4s
130:	test: 0.4772189	best: 0.4838545 (120)	total: 2.25s	remaining: 32.1s
1